In [ ]:
import hylite
from hylite import io
import numpy as np

import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline

Visualisation is an important part of checking, analysing and interpreting hyperspectral data. *hylite* uses matplotlib to rapidly generate visualisations that are highly customisable. While this can take a little getting used to, it allows production-ready figures to be created with a little experience. 

The following notebook shows some examples of how *hylite* can be used to create different false-colour visualisations of hyperspectral data. Subsequent notebooks show different methods for visualising more advanced analyses (e.g. minimum wavelength maps).

In [ ]:
# load a dataset to work with
cloud = io.load( 'test_data/hypercloud.ply' )
cloud.decompress() # this was compressed from float to integer to save space; so we need to convert it back

# convert it to an image for convenience (saves us rendering it every time)
image = cloud.render( cloud.header.get_camera(0), (0,-1) ) # push point spectra onto image

### 1. Quick_plot

The quick plot function has a variety of settings that can be used to achieve many different visualisations. These include:
 * bfac, cfac = fudge-factors to tweak image brightness; output pixels are scaled using out = pixel*(1+cfac) + bfac
 * vmin, vmax = set the colour stretch. Integers are interpreted as percentiles, floats as defined clipping values.
 * tscale = Normalise bands independently if True. By default clipping is applied to the whole dataset.
 * rot, flipX, flipY = convenient flags for rotating the image by 90 degrees or flipping it.
 
Lets start by exploring the role of cfac and bfac:

In [ ]:
fig,ax = plt.subplots( 3,3, figsize=(10,5))

for v,a in zip([-0.3,0.0,0.3], ax):
    for v2,_a in zip([0,0.25,0.5],a):
        image.quick_plot(hylite.SWIR, ax=_a, cfac=v2, bfac=v )
        _a.set_title("bfac = %.2f, cfac = %.2f" % (v,v2) )
fig.tight_layout()
fig.show()

Similar changes in brightness/contrast can be achieved by changing the colour stretch (vmin and vmax values):

In [ ]:
fig,ax = plt.subplots( 2,3, figsize=(12,5))

for v,a in zip([False, True], ax):
    for v2,_a in zip([0,5,10],a):
        image.quick_plot(hylite.SWIR, ax=_a, vmin=v2, vmax=100-v2, tscale=v )
        _a.set_title("clip = %d-%d (per band = %s)" % (v2,100-v2, v) )
fig.tight_layout()
fig.show()

### 2. Filtering and hole-filling

Image data can also be quickly adjusted to reduce noise (median filter) or fill gaps (nearest neighbour) for plotting purposes.

In [ ]:
image2 = image.copy() # avoid changing original image

fig,ax = plt.subplots( 1,3, figsize=(15,5))
image2.quick_plot(hylite.SWIR, ax=ax[0])
image2.fill_holes() # replace small holes with neighbour values
image2.set_as_nan(0)
image2.quick_plot(hylite.SWIR, ax=ax[1])
image2.despeckle() # apply median filter smoothing
image2.quick_plot(hylite.SWIR, ax=ax[2])

for a,t in zip( ax,['Original', 'Holes filled', 'Despeckled'] ):
    a.set_title(t)
fig.tight_layout()
fig.show()

### Saturation enhancement

For many visualisations, it can be useful to tweak the hue, saturation or value of the resulting data. *hylite* has some basic tools for doing this, and for more complex approaches matplotlib has several useful functions ( e.g., *rgb_to_hsv* and *hsv_to_rgb* ).

In [ ]:
from hylite.filter import boost_saturation

enh = boost_saturation( image2, hylite.SWIR,
                     flip = True, # invert colours [can make sense if high values = deep absorbtions]
                     sat=0.5, # fix saturation
                     val=0.5, # fix brightness
                     clip = (1,99), # colour stretch
                     per_band=True, # apply clip per band
                     )

# remove background 
enh.data[ np.isnan( image2.data ).all(axis=-1), : ] = np.nan

fig,ax = enh.quick_plot((0,1,2))
fig.show()

### Saving figures

While matplotlib figures can be easily saved using *fig.save_fig(...)*, it is often useful to save visualisations at their original resolution. This can be done in several ways:

###### Using quick_plot

In [ ]:
fig,ax = image.quick_plot(hylite.SWIR, path='./outputs/myimage.png')
plt.clf()

##### Using export_bands

In [ ]:
rgb = image.export_bands(hylite.SWIR)
#rgb.data = rgb.data[:,:,::-1] # N.B. for true colour (RGB) we would need to flip the order here, as they were exported in order of wavelength (b,g,r)
rgb.percent_clip(5,95) # apply a percentile clip
io.save('./outputs/myotherimage.png', rgb )

In [ ]:
io.save('./outputs/andanotherimage.png', enh ) # even easier for ternary images!